In [51]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [52]:
ws = Workspace.from_config()

In [53]:
ws.get_default_datastore()

In [54]:
exp = Experiment(workspace=ws, name='grab-ai-200')

In [55]:
script_folder = '../remote_run'

In [56]:
ds = ws.get_default_datastore()

In [63]:
# ds.download(target_path='./data', overwrite=False, show_progress=True)

In [57]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target
{'creationTime': '2019-06-16T13:01:46.843268+00:00', 'vmPriority': 'Dedicated', 'currentNodeCount': 1, 'allocationStateTransitionTime': '2019-06-17T13:32:41.023000+00:00', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'nodeIdleTimeBeforeScaleDown': 'PT120S', 'maxNodeCount': 2}, 'modifiedTime': '2019-06-16T13:03:07.972287+00:00', 'errors': None, 'vmSize': 'STANDARD_NC6', 'provisioningState': 'Succeeded', 'targetNodeCount': 1, 'allocationState': 'Steady', 'nodeStateCounts': {'runningNodeCount': 1, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'preparingNodeCount': 0}}


In [58]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

cpu-cluster AmlCompute Succeeded
gpu-cluster AmlCompute Succeeded


In [59]:
from azureml.train.dnn import TensorFlow

script_params = {
    '--data-folder': ws.get_default_datastore().as_mount(),
    '--epoch': 200,
    '--batch-size': 16,
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='./remote_train_resnet.py', 
                 use_gpu=True,
                 pip_packages=['keras', 'matplotlib', 'pandas', 'Pillow', 'pyspark'],
                 framework_version='1.13')

In [60]:
run = exp.submit(est)

In [50]:
from azureml.widgets import RunDetails

RunDetails(run).show()

_UserRunWidget(widget_settings={'send_telemetry': False, 'sdk_version': '1.0.43', 'childWidgetDisplay': 'popup…

In [21]:
ds = ws.get_default_datastore()
ds.download(target_path='./outputs', prefix='./outputs/', overwrite=False, show_progress=True)

0